In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import pandas as pd
import datasets

In [3]:
dataset = datasets.load_dataset('snli')
dataset

Reusing dataset snli (/home/richiew/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
})

In [4]:
prepare_train_dataset = prepare_eval_dataset = \
            lambda exs: prepare_dataset_nli(exs, tokenizer, args.max_length)

In [5]:
train_dataset = dataset['train']

In [6]:
from nltk.corpus import stopwords
import string

stop = stopwords.words('english')
trans = str.maketrans('', '', string.punctuation)
def remove_stop_count_words(df):
    df['premise_wo_stopwords'] = df['premise'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]).strip().lower().translate(trans))
    return df.premise_wo_stopwords.str.split(expand=True).stack().value_counts()

train_df = train_dataset.to_pandas()

entail_df = train_df[train_df.label == 0]
neutral_df = train_df[train_df.label == 1]
contradict_df = train_df[train_df.label == 2]

In [7]:
entail_counts = remove_stop_count_words(entail_df).reset_index()
entail_counts

/tmp/ipykernel_1193/2886799954.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['premise_wo_stopwords'] = df['premise'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]).strip().lower().translate(trans))


,index,0
0,a,103933
1,man,50342
2,woman,26126
3,two,25263
4,wearing,20078
...,...,...
19671,pocketbooks,1
19672,elevates,1
19673,headtotoe,1
19674,illusion,1


In [8]:
neutral_counts = remove_stop_count_words(neutral_df).reset_index()
neutral_counts

/tmp/ipykernel_1193/2886799954.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['premise_wo_stopwords'] = df['premise'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]).strip().lower().translate(trans))


,index,0
0,a,103611
1,man,50143
2,woman,26108
3,two,25114
4,people,20068
...,...,...
19624,nemo,1
19625,laundered,1
19626,joys,1
19627,lecturers,1


In [9]:
contradict_counts = remove_stop_count_words(contradict_df).reset_index()
contradict_counts

/tmp/ipykernel_1193/2886799954.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['premise_wo_stopwords'] = df['premise'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]).strip().lower().translate(trans))


,index,0
0,a,103847
1,man,50285
2,woman,26117
3,two,25209
4,wearing,20054
...,...,...
19679,craftslady,1
19680,gasping,1
19681,gasp,1
19682,portopotties,1


In [10]:
entail_counts[~entail_counts.index.isin(contradict_counts.index)].dropna()

,index,0


In [11]:
entail_counts[~entail_counts.index.isin(neutral_counts.index)].dropna()

,index,0
19629,alsosleeping,1
19630,expats,1
19631,composure,1
19632,regains,1
19633,profusely,1
19634,4315,1
19635,complexioned,1
19636,guidebooks,1
19637,cataloging,1
19638,dayhiker,1


In [12]:
contradict_counts[~contradict_counts.index.isin(entail_counts.index)].dropna()

,index,0
19676,clutter,1
19677,lawns,1
19678,kentucky,1
19679,craftslady,1
19680,gasping,1
19681,gasp,1
19682,portopotties,1
19683,nailgloss,1


In [13]:
import nltk
from nltk.corpus import words
nltk.download('words')
words_set = list(set([x.lower() for x in words.words()]))
words_df = pd.DataFrame()
words_df['words'] = words_set
words_df

[nltk_data] Downloading package words to /home/richiew/nltk_data...
[nltk_data]   Package words is already up-to-date!


,words
0,expanded
1,usedly
2,equalable
3,rotter
4,phototelescopic
...,...
234372,strickenly
234373,falconbill
234374,zoophaga
234375,squirk


In [14]:
words_df[~words_df.words.isin(contradict_counts['index'])].dropna()

,words
0,expanded
1,usedly
2,equalable
3,rotter
4,phototelescopic
...,...
234372,strickenly
234373,falconbill
234374,zoophaga
234375,squirk


In [15]:
words_df[~words_df.words.isin(entail_counts['index'])].dropna().to_csv('entail-missing-words.csv')
words_df[~words_df.words.isin(entail_counts['index'])].dropna()

,words
0,expanded
1,usedly
2,equalable
3,rotter
4,phototelescopic
...,...
234372,strickenly
234373,falconbill
234374,zoophaga
234375,squirk


In [16]:
"""
Words that I might be able to use:
liquer
decompress
feminize
airway
uniformness
guesswork
decently
courteous
energetics
biomathematics
commentary
interpreter
parallelizer
unequitably
doghood
dutiful
slabbed
liberative
nondivisible
alt
hungerless
encrypt
unproportional
brainsickness
forgetting
squeaky
fad
radiate
hoarder
earful

Used:
dowery
criteria
surfacing
upbeat
unagressive
emotionless
biomathematics
airway 
squeaky
encrypt
interpreter
commentary

entailment examples:

A bride's family gifts the husband money in exchange for marriage
The union had the criteria of a dowery

The vessel rises above the ocean
A submarine sufacing makes a big splash

A group of friends dance to music
An upbeat song is playing for people

A security guard calmly escorts the person off the property
The patrol is unaggressive

The girl stares off into the sunset in thought
Someone appears emotionless as the day ends

The graduate holds a biomathematics degree on stage
He is ready to work as a medical researcher
"""



"\nWords that I might be able to use:\nliquer\ndecompress\nfeminize\nairway\nuniformness\nguesswork\ndecently\ncourteous\nenergetics\nbiomathematics\ncommentary\ninterpreter\nparallelizer\nunequitably\ndoghood\ndutiful\nslabbed\nliberative\nnondivisible\nalt\nhungerless\nencrypt\nunproportional\nbrainsickness\nforgetting\nsqueaky\nfad\nradiate\nhoarder\nearful\n\nUsed:\ndowery\ncriteria\nsurfacing\nupbeat\nunagressive\nemotionless\nbiomathematics\nairway \nsqueaky\nencrypt\ninterpreter\ncommentary\n\nentailment examples:\n\nA bride's family gifts the husband money in exchange for marriage\nThe union had the criteria of a dowery\n\nThe vessel rises above the ocean\nA submarine sufacing makes a big splash\n\nA group of friends dance to music\nAn upbeat song is playing for people\n\nA security guard calmly escorts the person off the property\nThe patrol is unaggressive\n\nThe girl stares off into the sunset in thought\nSomeone appears emotionless as the day ends\n\nThe graduate holds a bi

In [17]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json
import pickle

def eval_model(dataset):
    with open('eval_args.pkl', 'rb') as filehandler:
        training_args = pickle.load(filehandler)
    with open('args.pkl', 'rb') as filehandler:
        args = pickle.load(filehandler)

    default_datasets = {'qa': ('squad',), 'nli': ('snli',)}
    dataset_id = tuple(args.dataset.split(':')) if args.dataset is not None else \
        default_datasets[args.task]
    # MNLI has two validation splits (one with matched domains and one with mismatched domains). Most datasets just have one "validation" split
    eval_split = 'validation_matched' if dataset_id == ('glue', 'mnli') else 'validation'

    # NLI models need to have the output label count specified (label 0 is "entailed", 1 is "neutral", and 2 is "contradiction")
    task_kwargs = {'num_labels': 3} if args.task == 'nli' else {}

    # Here we select the right model fine-tuning head
    model_classes = {'qa': AutoModelForQuestionAnswering,
                        'nli': AutoModelForSequenceClassification}
    model_class = model_classes[args.task]
    # Initialize the model and tokenizer from the specified pretrained model/checkpoint
    model = model_class.from_pretrained(args.model, **task_kwargs)
    tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=True)

    # Select the dataset preprocessing function (these functions are defined in helpers.py)
    if args.task == 'qa':
        prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
        prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)
    elif args.task == 'nli':
        prepare_train_dataset = prepare_eval_dataset = \
            lambda exs: prepare_dataset_nli(exs, tokenizer, args.max_length)
        # prepare_eval_dataset = prepare_dataset_nli
    else:
        raise ValueError('Unrecognized task name: {}'.format(args.task))

    print("Preprocessing data... (this takes a little bit, should only happen once per dataset)")
    if dataset_id == ('snli',):
        # remove SNLI examples with no label
        dataset = dataset.filter(lambda ex: ex['label'] != -1)

    eval_dataset = None
    train_dataset_featurized = None
    eval_dataset_featurized = None
    
    if training_args.do_eval:
        eval_dataset = dataset[eval_split]
        if args.max_eval_samples:
            eval_dataset = eval_dataset.select(range(args.max_eval_samples))
        eval_dataset_featurized = eval_dataset.map(
            prepare_eval_dataset,
            batched=True,
            num_proc=2,
            remove_columns=eval_dataset.column_names
        )

    # Select the training configuration
    trainer_class = Trainer
    eval_kwargs = {}
    # If you want to use custom metrics, you should define your own "compute_metrics" function.
    # For an example of a valid compute_metrics function, see compute_accuracy in helpers.py.
    compute_metrics = None
    if args.task == 'qa':
        # For QA, we need to use a tweaked version of the Trainer (defined in helpers.py)
        # to enable the question-answering specific evaluation metrics
        trainer_class = QuestionAnsweringTrainer
        eval_kwargs['eval_examples'] = eval_dataset
        metric = datasets.load_metric('squad')
        compute_metrics = lambda eval_preds: metric.compute(
            predictions=eval_preds.predictions, references=eval_preds.label_ids)
    elif args.task == 'nli':
        compute_metrics = compute_accuracy


    # This function wraps the compute_metrics function, storing the model's predictions
    # so that they can be dumped along with the computed metrics
    eval_predictions = None
    def compute_metrics_and_store_predictions(eval_preds):
        nonlocal eval_predictions
        eval_predictions = eval_preds
        return compute_metrics(eval_preds)

    # Initialize the Trainer object with the specified arguments and the model and dataset we loaded above
    trainer = trainer_class(
        model=model,
        args=training_args,
        train_dataset=train_dataset_featurized,
        eval_dataset=eval_dataset_featurized,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_and_store_predictions
    )
    # Train and/or evaluate
    if training_args.do_train:
        trainer.train()
        trainer.save_model()
        # If you want to customize the way the loss is computed, you should subclass Trainer and override the "compute_loss"
        # method (see https://huggingface.co/transformers/_modules/transformers/trainer.html#Trainer.compute_loss).
        #
        # You can also add training hooks using Trainer.add_callback:
        #   See https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer.add_callback
        #   and https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerCallback

    if training_args.do_eval:
        results = trainer.evaluate(**eval_kwargs)

        # To add custom metrics, you should replace the "compute_metrics" function (see comments above).
        #
        # If you want to change how predictions are computed, you should subclass Trainer and override the "prediction_step"
        # method (see https://huggingface.co/transformers/_modules/transformers/trainer.html#Trainer.prediction_step).
        # If you do this your custom prediction_step should probably start by calling super().prediction_step and modifying the
        # values that it returns.

        print('Evaluation results:')
        print(results)

        os.makedirs(training_args.output_dir, exist_ok=True)

        with open(os.path.join(training_args.output_dir, 'eval_metrics.json'), encoding='utf-8', mode='w') as f:
            json.dump(results, f)

        with open(os.path.join(training_args.output_dir, 'eval_predictions.jsonl'), encoding='utf-8', mode='w') as f:
            if args.task == 'qa':
                predictions_by_id = {pred['id']: pred['prediction_text'] for pred in eval_predictions.predictions}
                for example in eval_dataset:
                    example_with_prediction = dict(example)
                    example_with_prediction['predicted_answer'] = predictions_by_id[example['id']]
                    f.write(json.dumps(example_with_prediction))
                    f.write('\n')
            else:
                for i, example in enumerate(eval_dataset):
                    example_with_prediction = dict(example)
                    example_with_prediction['predicted_scores'] = eval_predictions.predictions[i].tolist()
                    example_with_prediction['predicted_label'] = int(eval_predictions.predictions[i].argmax())
                    f.write(json.dumps(example_with_prediction))
                    f.write('\n')

In [18]:
from datasets import Dataset, DatasetDict
custom_datadict = DatasetDict()
custom_data = Dataset.from_json('custom_data/custom_eval.json')
custom_datadict['validation'] = custom_data
custom_datadict

Using custom data configuration default-d8a9957078c9c74f
Reusing dataset json (/home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0)


DatasetDict({
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10
    })
})

In [19]:
eval_model(custom_datadict)

Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0/cache-06c7867477a0d4d9.arrow


Preprocessing data... (this takes a little bit, should only happen once per dataset)


Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0/cache-8fbe281e6c09b00f.arrow
Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0/cache-33de89b7c83ab5f2.arrow
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


Evaluation results:
{'eval_loss': 1.2181907892227173, 'eval_accuracy': 0.5, 'eval_runtime': 0.0589, 'eval_samples_per_second': 169.646, 'eval_steps_per_second': 33.929}


In [20]:
def featurize_data(dataset):
    with open('eval_args.pkl', 'rb') as filehandler:
        training_args = pickle.load(filehandler)
    with open('args.pkl', 'rb') as filehandler:
        args = pickle.load(filehandler)

    default_datasets = {'qa': ('squad',), 'nli': ('snli',)}
    dataset_id = tuple(args.dataset.split(':')) if args.dataset is not None else \
        default_datasets[args.task]
    # MNLI has two validation splits (one with matched domains and one with mismatched domains). Most datasets just have one "validation" split
    eval_split = 'validation_matched' if dataset_id == ('glue', 'mnli') else 'validation'

    # NLI models need to have the output label count specified (label 0 is "entailed", 1 is "neutral", and 2 is "contradiction")
    task_kwargs = {'num_labels': 3} if args.task == 'nli' else {}

    # Here we select the right model fine-tuning head
    model_classes = {'qa': AutoModelForQuestionAnswering,
                        'nli': AutoModelForSequenceClassification}
    model_class = model_classes[args.task]
    # Initialize the model and tokenizer from the specified pretrained model/checkpoint
    model = model_class.from_pretrained(args.model, **task_kwargs)
    tokenizer = AutoTokenizer.from_pretrained(args.model, use_fast=True)

    # Select the dataset preprocessing function (these functions are defined in helpers.py)
    if args.task == 'qa':
        prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
        prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)
    elif args.task == 'nli':
        prepare_train_dataset = prepare_eval_dataset = \
            lambda exs: prepare_dataset_nli(exs, tokenizer, args.max_length)
        # prepare_eval_dataset = prepare_dataset_nli
    else:
        raise ValueError('Unrecognized task name: {}'.format(args.task))

    print("Preprocessing data... (this takes a little bit, should only happen once per dataset)")
    if dataset_id == ('snli',):
        # remove SNLI examples with no label
        dataset = dataset.filter(lambda ex: ex['label'] != -1)
    
    eval_dataset = dataset[eval_split]
    if args.max_eval_samples:
        eval_dataset = eval_dataset.select(range(args.max_eval_samples))
    eval_dataset_featurized = eval_dataset.map(
        prepare_eval_dataset,
        batched=True,
        num_proc=2,
        remove_columns=eval_dataset.column_names
    )

    return eval_dataset_featurized

In [46]:
from datasets import Dataset, DatasetDict

train_datadict = DatasetDict()
train_data = Dataset.from_json('custom_data/custom_train.json')
train_datadict['validation'] = train_data

test_datadict = DatasetDict()
test_data = Dataset.from_json('custom_data/custom_eval.json')
test_datadict['validation'] = test_data

feat_train_dataset = featurize_data(train_datadict)
feat_test_dataset = featurize_data(test_datadict)

snli_dataset = featurize_data(datasets.load_dataset('snli'))

Using custom data configuration default-3fb92396f0bc704f


0 tables [00:00, ? tables/s]

Using custom data configuration default-d8a9957078c9c74f
Reusing dataset json (/home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0)
loading configuration file ./trained_model/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",


Dataset json downloaded and prepared to /home/richiew/.cache/huggingface/datasets/json/default-3fb92396f0bc704f/0.0.0. Subsequent calls will reuse this data.
Preprocessing data... (this takes a little bit, should only happen once per dataset)


  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file ./trained_model/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transform

Preprocessing data... (this takes a little bit, should only happen once per dataset)


Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0/cache-33de89b7c83ab5f2.arrow
Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/json/default-d8a9957078c9c74f/0.0.0/cache-8fbe281e6c09b00f.arrow
Reusing dataset snli (/home/richiew/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
loading configuration file ./trained_model/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1":

Preprocessing data... (this takes a little bit, should only happen once per dataset)


Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-8ce2253536cb23ce.arrow
Loading cached processed dataset at /home/richiew/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-8a38803079cdfd6c.arrow


In [52]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Setting the Parameters of the Model
param = {'eta': 0.3,
         'max_depth': 5,
         'objective': 'binary:logistic',
         'learning_rate': 0.1}

xgb_train_labels = [1 if x == 0 else 0 for x in feat_train_dataset['label']]
xgb_test_labels = [1 if x == 0 else 0 for x in feat_test_dataset['label']]

xgb_train_inputs = xgb.DMatrix(feat_train_dataset['input_ids'], label=xgb_train_labels)
xgb_test_inputs = xgb.DMatrix(feat_test_dataset['input_ids'], label=xgb_test_labels)

# Training the Model
xgb_model = xgb.train(param, xgb_train_inputs, 3)

# Predicting using the Model
y_pred = xgb_model.predict(xgb_test_inputs)
print(y_pred)
y_pred = np.where(np.array(y_pred) > 0.5, 1, 0)

# Evaluation of Model
print('Acc: ', accuracy_score(xgb_test_labels, y_pred))
print('F1: ', f1_score(xgb_test_labels, y_pred))

[16:44:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.5554362  0.53062665 0.42164522 0.5554362  0.53062665 0.53062665
 0.51278526 0.42164522 0.51278526 0.42164522]
Acc:  0.7
F1:  0.8235294117647058
